## Этап 2. Анализ и очистка витрины квартир `flats_features`

В этом ноутбуке:
- подключаемся к персональной БД (`destination_db`),
- загружаем витрину `flats_features`,
- проводим базовый анализ качества данных (размер, статистики, категории),
- применяем функции очистки (дубликаты, выбросы, пропуски),
- сравниваем датасет **до** и **после** очистки.

Этот ноутбук можно показывать как основной артефакт для проверки этапа 2.


In [ ]:
import os
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Подгружаем .env из корня part1_airlfow
env_path = Path.cwd().parent / ".env"
load_dotenv(env_path)

dst_host = os.environ.get("DB_DESTINATION_HOST")
dst_port = os.environ.get("DB_DESTINATION_PORT")
dst_username = os.environ.get("DB_DESTINATION_USER")
dst_password = os.environ.get("DB_DESTINATION_PASSWORD")
dst_db = os.environ.get("DB_DESTINATION_NAME")

print(f"Подключаюсь к базе: {dst_db}")
print(f"Хост: {dst_host}")

conn = create_engine(
    f"postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}"
)


In [ ]:
def load_flats_features() -> pd.DataFrame:
    """Загружает витрину flats_features из персональной БД."""
    exists_query = """
        SELECT EXISTS (
            SELECT 1
            FROM information_schema.tables
            WHERE table_schema = 'public'
              AND table_name = 'flats_features'
        )
    """
    exists = pd.read_sql(exists_query, conn)
    if not bool(exists.iloc[0, 0]):
        raise RuntimeError("Таблица flats_features не найдена в базе данных")

    df = pd.read_sql("SELECT * FROM flats_features", conn)
    print(f"Исходный датасет flats_features: shape = {df.shape}")
    display(df.head())
    return df


In [ ]:
def explore_categorical(df: pd.DataFrame) -> None:
    """Быстрый обзор категориальных признаков витрины квартир."""
    features = df.drop(columns=["id"], errors="ignore")
    cat_features = features.select_dtypes(include=["object"])

    if cat_features.empty:
        print("Категориальных признаков не найдено.")
        return

    unique_counts = cat_features.nunique()
    print("Число уникальных значений по категориальным признакам:")
    display(unique_counts.sort_values())
    print()

    distribution = unique_counts.value_counts().sort_index()
    print("Распределение признаков по количеству уникальных значений:")
    display(distribution)

    # Для признаков с небольшим числом уникальных значений
    small_card_cols = unique_counts[unique_counts <= 10].index.tolist()
    if small_card_cols:
        print("\nПримеры распределений для признаков с <= 10 уникальными значениями:")
        for col in small_card_cols:
            print(f"\n{col}:")
            print(cat_features[col].value_counts(dropna=False))


In [ ]:
def remove_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    """Удаляет дубликаты по идентификатору квартиры id."""
    before = len(df)
    df_clean = df.drop_duplicates(subset=["id"], keep="first")
    after = len(df_clean)
    print(f"Дубликаты по id: удалено {before - after} строк")
    return df_clean


def remove_outliers_iqr(df: pd.DataFrame, cols) -> pd.DataFrame:
    """Удаляет выбросы по IQR для указанных числовых признаков."""
    df_clean = df.copy()
    for col in cols:
        if col not in df_clean.columns:
            continue
        q1 = df_clean[col].quantile(0.25)
        q3 = df_clean[col].quantile(0.75)
        iqr = q3 - q1
        lb = q1 - 1.5 * iqr
        ub = q3 + 1.5 * iqr
        before = len(df_clean)
        df_clean = df_clean[(df_clean[col] >= lb) & (df_clean[col] <= ub)]
        after = len(df_clean)
        print(f"Выбросы по {col}: удалено {before - after} строк")
    return df_clean


def fill_missing(df: pd.DataFrame) -> pd.DataFrame:
    """Заполняет пропуски: числовые — медиана, категориальные — мода, булевые — False."""
    df_clean = df.copy()

    num_cols = df_clean.select_dtypes(include=["int64", "float64"]).columns
    cat_cols = df_clean.select_dtypes(include=["object"]).columns
    bool_cols = df_clean.select_dtypes(include=["bool"]).columns

    for col in num_cols:
        med = df_clean[col].median()
        df_clean[col] = df_clean[col].fillna(med)

    for col in cat_cols:
        if df_clean[col].isna().any():
            mode = df_clean[col].mode()
            if len(mode) > 0:
                df_clean[col] = df_clean[col].fillna(mode.iloc[0])
            else:
                df_clean[col] = df_clean[col].fillna("Unknown")

    for col in bool_cols:
        df_clean[col] = df_clean[col].fillna(False)

    print("Пропуски после заполнения:")
    display(df_clean.isna().sum())
    return df_clean


def clean_flats_features(df: pd.DataFrame) -> pd.DataFrame:
    """Полный пайплайн очистки flats_features."""
    print("\n=== Старт очистки flats_features ===")
    df1 = remove_duplicates(df)
    df2 = remove_outliers_iqr(df1, ["price", "total_area", "kitchen_area", "living_area"])
    df3 = fill_missing(df2)
    print(f"\nРазмер до очистки: {df.shape}, после очистки: {df3.shape}")
    return df3


In [ ]:
# Загрузка исходных данных и базовый анализ
raw = load_flats_features()

print("\nСтатистика до очистки (price, total_area, kitchen_area, living_area):")
display(raw[["price", "total_area", "kitchen_area", "living_area"]].describe())

print("\nОбзор категориальных признаков:")
explore_categorical(raw)


In [ ]:
# Очистка датасета и сравнение статистик
cleaned = clean_flats_features(raw)

print("\nСтатистика после очистки (price, total_area, kitchen_area, living_area):")
display(cleaned[["price", "total_area", "kitchen_area", "living_area"]].describe())


In [ ]:
# Опционально: закрываем соединение с БД
conn.dispose()
